In [ ]:
cd ../

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd


train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

In [ ]:
from data_utils import get_col_names


cols = get_col_names("_strain")
cols

In [ ]:
strains = set()

for col in cols:
    strains.update(train_metadata[col].dropna().unique())
    
strains

In [ ]:
from collections import defaultdict
import pandas as pd

unique_strains = defaultdict(set)

# get unique strains for all cols by lab

for lab_id, grp in train_metadata.groupby("lab_id"):

    for col in cols:
        unique_strains[lab_id].update(grp[col].dropna().unique())
        
pd.DataFrame({
    "lab_id": list(unique_strains.keys()),
    "unique_strains": [list(s) for s in unique_strains.values()]
})

In [ ]:
# unique strains for all mouses by labs

strains_by_labs = defaultdict(lambda: defaultdict(set))

for lab_id, grp in train_metadata.groupby("lab_id"):
    
    for col in cols:
        strains_by_labs[lab_id][col].update(grp[col].dropna().unique())
        
for lab_id, strains in strains_by_labs.items():
    print(lab_id)

    for col, strain in strains.items():
            print(f"{col}: {strain}")

    print()

In [ ]:
for idx, row in train_metadata.iterrows():
    print(f"{row[cols[0]]} || {row[cols[1]]} || {row[cols[2]]} || {row[cols[3]]}")

In [68]:
# count of each unique strain global


counts = (
    train_metadata[cols]
    .stack(dropna=True)            # long format, drops NA
    .value_counts()                # frequency per unique strain
    .sort_values(ascending=False)
)
print(counts)


C57Bl/6J            14800
BTBR                 9786
BALB/c                606
CD1                   116
C57Bl/6N               89
CD-1 (ICR)             61
CFW                    38
C57Bl/6J x Ai148       24
129/SvEvTac            11
Name: count, dtype: int64


/var/folders/nh/9jyn0n394vxccrbpw3r3t5940000gn/T/ipykernel_12785/3503188329.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(dropna=True)            # long format, drops NA


In [67]:
import pandas as pd

# cols are the strain columns you want to aggregate (e.g., ["mouse1_strain","mouse2_strain",...])
melted = train_metadata.melt(
    id_vars="lab_id",
    value_vars=cols,
    var_name="mouse_attr",
    value_name="strain",
)

# drop missing strains, then count per (lab_id, strain)
lab_strain_counts = (
    melted.dropna(subset=["strain"])
          .groupby(["lab_id", "strain"])
          .size()
          .reset_index(name="count")
          .sort_values(["lab_id", "count"], ascending=[True, False])
)

# Optional: wide matrix of counts with labs as rows and strains as columns
lab_strain_matrix = (
    melted.dropna(subset=["strain"])
          .groupby(["lab_id", "strain"])
          .size()
          .unstack(fill_value=0)
          .sort_index()
)



In [65]:
lab_strain_matrix

strain,129/SvEvTac,BALB/c,BTBR,C57Bl/6J,C57Bl/6J x Ai148,C57Bl/6N,CD-1 (ICR),CD1,CFW
lab_id,,,,,,,,,
AdaptableSnail,0,0,0,0,0,0,61,0,0
BoisterousParrot,0,0,0,0,0,16,0,0,0
CRIM13,0,21,0,21,0,0,0,0,0
CalMS21_supplemental,0,297,0,297,0,0,0,0,0
CalMS21_task1,0,101,0,101,0,0,0,0,0
CalMS21_task2,0,76,0,76,0,0,0,0,0
CautiousGiraffe,0,0,0,20,0,0,0,0,0
DeliriousFly,0,6,0,6,0,0,0,0,0
ElegantMink,11,0,0,27,0,0,0,0,0
